<a href="https://colab.research.google.com/github/felipedram/xf-models-analysis/blob/master/trabalho_final.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

[Funções para teste](http://delta.cs.cinvestav.mx/~ccoello/EMOO/testfuncs/)<br>
[Documentação basin-hopping](https://docs.scipy.org/doc/scipy/reference/generated/scipy.optimize.basinhopping.html#scipy.optimize.basinhopping)<br>
[Documentação minimize](https://docs.scipy.org/doc/scipy/reference/generated/scipy.optimize.minimize.html)<br>
[Basin-Hopping Paper](https://arxiv.org/pdf/cond-mat/9803344.pdf)<br>
[Stochastic Gradient Descent Towards Data Science](https://towardsdatascience.com/gradient-descent-in-python-a0d07285742f)<br>
[Stochastic Gradient Descent](https://www.pyimagesearch.com/2016/10/17/stochastic-gradient-descent-sgd-with-python/)
[StatQuest Gradient Descent](https://www.youtube.com/watch?v=sDv4f4s2SB8)<br>
[StatQuest Stochastic Gradient Descent](https://www.youtube.com/watch?v=vMh0zPT0tLI)

Em \cite{Ekel2002} foram introduzidos os modelos $\langle X,R \rangle$ e $\langle X,F \rangle$ como uma forma de lidar com as incertezas inerentes ao processo de tomada de decisão multicritério, utilizando a teoria de conjuntos fuzzy. O primeiro é aplicado a problemas multicritério e, o segundo, a problemas multiobjetivo. Para análise dos modelos, o autor utilizara a abordagem Bellman-Zadeh, técnicas de relação de preferência *fuzzy* e introduziu o conceito de solução harmoniosa. Nas seções a seguir, vamos explorar o modelo $\langle X,F \rangle$, falando sobre as motivações que levaram a sua proposição, discutindo suas características e exemplificando sua aplicação.

## Introdução

### Otimização multiobjetivo

De maneira bastante rasa, estamos diante de um problema de otimização matemática quando temos uma função $f(X)$ e queremos encontrar o $X$ que *maximiza* ou *minimiza* seu resultado. À essa função, chamamos de Função Objetivo (FO) e, matematicamente, escrevemos

$$
\begin{equation}
    f(X) \rightarrow extr,
\end{equation}
$$
onde extr significa extremo, indicando que queremos levar essa função a um dos extremos (máximo ou mínimo).

Normalmente, esse tipo de problema possui restrições, de tal maneira que não pode ser aceito qualquer valor de $X$. Assim, aos valores aceitáveis para $X$, dá-se o nome de conjunto das soluções permissíveis, ou região das soluções factíveis, ou espaço de busca, ou $L$, simplesmente. Dentre os possíveis valores de $X$ ($X \in L$), àquele que leva $f(X)$ ao extemo, chamamos de $X^*$ (alguns autores também chamam de $X^0$).

Agora imagine um problema em que, ao invés de uma, temos algumas (2, 3, 4, ...) FOs. Ou seja, temos um conjunto (ou vetor) $F(X) = \{f_0(X), f_1(X), f_2(X), ...\}$ e queremos encontrar $X^*$. Ou seja, um único $X$ que maximiza (ou minimiza) todas as FOs ao mesmo tempo e que, assim como no primeiro problema, está sujeito a uma série de regras/restrições. Este é um problema de *otimização multiobjetivo* que escrevemos como

$$
\begin{equation}
    F(X) \rightarrow extr \text{.}
\end{equation}
$$

Certamente, encontrar esse $X^*$ não é tarefa fácil, principalmente porque, normalmente, as FOs são conflitantes. Então, há que se discutir questões como normalização, princípios de otimalidade e prioridades dos critérios. A solução dessas questões passa por técnicas de escalarização, imposição das restrições, método da função utilidade, programação de metas e utilização do princípio da garantia do resultado. O autor de \cite{Ekel2002}, indica como muito importante uma especial atenção para a utiliação do princípio da garantia do resultado, assim como uma lacuna na definição de "solução ótima". Para tratar dessas questões, Ekel propôs a aplicação da abordagem Bellman-Zadeh. Nas seções a seguir vamos implementar e discutir algumas aplicações da abordagem proposta em XX, bem como alguns prós e pontos de atenção para utilização do método.

### Modelo $\langle X,F \rangle$ e a Abordagem de Bellman-Zadeh

Em problemas de otimização multiobjetivo, ou problemas do tipo (2), não existe uma única solução que otimize a todos os objetivos simultaneamente. Nesses casos, falamos em soluções ótimas de Pareto: quando não é mais possível melhorar o valor de uma FO sem degradar o valor das demais e/ou sem acréscimo de novas informações. Essas soluções também são conhecidas como soluções não-dominadas, ótimas de Pareto, Pareto eficientes ou não-inferiores. Todas as soluções da *fronteira de Pareto*, que aqui chamaremos de $\Omega$, são consideradas igualmente boas. Agora, imagine que você é um profissional contratado para resolver um problema de otimização multiobjetivo. O que seu cliente diria se, como resultado de meses de trabalho, você entregasse a ele um conjunto infinito (ou finito incontável) de soluções?

Foi pensando em tratar essa questão, sem descartar a necessidade de se observar todas as outras citadas anteriormente (normalização, otimalidade, etc.), que Ekel propôs o modelo $\langle X,F \rangle$ e a utilização da abordagem Bellman-Zadeh para sua análise. A ideia central da proposta é associar, para cada FO, um número *fuzzy* $\{ X, \mu_{A}(X) \}$. Esse número *fuzzy* funciona como uma "nota" para cada $X \in L$ e indica o grau de pertinência desse $X$ à melhor solução possível para aquela FO --- ou o grau de pertinência de $X$ ao $extr$ daquela FO.

Como função de pertinência, ou seja, para o cálculo dessa nota, Ekel propôs

$$
	\begin{equation}
		\mu_{A_{p}}(X)=\Bigg[\frac{f_p(X)-\underset{X \in L}\min{f_p(X)}}{\underset{X \in L}\max{f_p(X)}-\underset{X \in L}\min{f_p(X)}}\Bigg]
	\end{equation}
$$
para FO a serem maximizadas e
$$
	\begin{equation}
		\mu_{A_{p}}(X)=\Bigg[\frac{\underset{X \in L}\max{f_p(X)}-f_p(X)}{\underset{X \in L}\max{f_p(X)}-\underset{X \in L}\min{f_p(X)}}\Bigg]
	\end{equation}
$$
para FO a serem minimizadas. Ekel também definiu que, dado um vetor $F$ de FOs, seus respectivos $\mu_A$ formam um vetor $\mu_D$.

Dito isso, imagine que temos um problema com 4 FOs. Utilizando a notação de (2), temos: $F(X) = \{f_0(X), f_1(X), f_2(X), f_3(X)\}$ --- vamos começar a contar em zero para nos adequarmos à linguagem Python. Se um ponto $X_i$ qualquer atende às FOs em um nível $\mu_D(X_i) = \{0.3, 0.5, 0.7, 0.4\}$ e outro ponto $X_j$ atende às FOs em um nível $\mu_D(X_j) = \{0.4, 0.5, 0.7, 0.6\}$, podemos dizer que o ponto $X_j$ é uma solução melhor pois o menor nível de pertinência de $X_j$ é maior que o menor nível de pertinência de $X_i$, ou $\min(\mu_D(X_j)) > \min(\mu_D(X_i))$. Olhando dessa forma, fica claro que um problema de otimização multiobjetivo foi transformado em um problema $\max \min$ pois o $X^*$ que buscamos agora é aquele que ofereça um $\mu_D$ cujo menor nível de pertinência é o maior possível. Também podemos dizer que essa forma de representar o problema nos permite resolver um problema multiobjetivo através de algoritmo monoobjetivo pois o algoritmo terá que trabalhar apenas com a função $\max \mu_D$. Isso ficará mais claro durante a iplementação computacional.

#### Implementação computacional

##### Representação do problema

Agora que introduzimos o modelo $\langle X,F \rangle$ e como analisá-lo, vamos fazer uma implementação computacional com dois exemplos. E começaremos por definir uma FO genérica:

In [0]:
import numpy as np

In [0]:
class f(object):
    """
    Função Objetivo genérica
    
    Classe que representa uma FO genérica, para implementação do modelo <X,F>
    """

    def __init__(self, solver, goal, f_max=None, x_max=None, f_min=None, x_min=None):
        self.__solver = solver  # Equação da função
        if goal == "max" or goal == "min":  # Objetivo a ser maximizado ou minimizado
            self.__goal = goal
        else:
            raise Exception("goal deve ser 'max' ou 'min'.")
        self.__f_max = f_max  # Valoes a serem calculados para mu
        self.__x_max = x_max
        self.__f_min = f_min
        self.__x_min = x_min

    @property
    def goal(self):
        return self.__goal

    @property
    def f_max(self):
        return self.__f_max

    @f_max.setter
    def f_max(self, f_max):
        self.__f_max = f_max

    @property
    def x_max(self):
        return self.__x_max

    @x_max.setter
    def x_max(self, x_max):
        self.__x_max = x_max

    @property
    def f_min(self):
        return self.__f_min

    @f_max.setter
    def f_min(self, f_min):
        self.__f_min = f_min

    @property
    def x_min(self):
        return self.__x_min

    @x_min.setter
    def x_min(self, x_min):
        self.__x_min = x_min

    def solve(self, x, inv=False):  # Retorna valor da fução para um dado ponto X
        x = np.array(x)
        return self.__solver(x) * -1 if inv else self.__solver(x)

    def mu(self, x):  # Cálculo de mu, conforme definição
        x = np.array(x)
        if self.__goal == "max":
            return (self.solve(x) - self.__f_min) / (self.__f_max - self.__f_min)
        elif self.__goal == "min":
            return (self.__f_max - self.solve(x)) / (self.__f_max - self.__f_min)

**Nota** sobre o método solve: Por que pode ser necessário inverter o valor de $f(X)$?<br>
Quando temos em mãos um algoritmo para minimização mas estamos diante de um objetivo a ser maximizado (ou vice-versa), a seguinte transformação pode ser útil:
$$
    \begin{equation}
        z = \underset{X \in L}{\min} f(X) = -\underset{X \in L}{\max} -f(X)
    \end{equation}
$$

Para um primeiro exemplo, vamos definir também uma FO linear, que é uma FO genérica cuja equação é um produto escalar dos coeficientes pelas variáveis.

In [0]:
class f_linear(f):
    def __init__(self, coefs, goal):
        self.__coefs = np.array(coefs)
        f.__init__(self, lambda x: np.dot(self.__coefs, x), goal)

    @property
    def coefs(self):
        return self.__coefs

Vamos agora tratar de um exemplo: um problema de alocação de recursos com 4 FOs lineares

$$
    \begin{align}
        f_{0}(X)&= 0,06x_0+0,53x_1+0,18x_2+0,18x_3+0,06x_4 \, \rightarrow \, \max \\
        f_{1}(X)&= 25x_0+70x_1+60x_2+95x_3+45x_4 \, \rightarrow \, \max \\
        f_{2}(X)&= 32,5x_1+300x_2+120x_3 \, \rightarrow \, \min \\
        f_{3}(X)&= 0,1x_0+0,1x_1+0,11x_2+0,35x_3+0,33x_4 \, \rightarrow \, \min
    \end{align}
$$

sujeitas às seguintes restrições

$$
\begin{align}
	0 \leq x_1 & \leq 850 \\
%
	0 \leq x_2 & \leq 220 \\
%
	0 \leq x_3 & \leq 1300 \\
%
	0 \leq x_4 & \leq 1615 \\
%
	0 \leq x_5 & \leq 700 \\
%
	x_1+x_2+x_3+x_4+x_5 & = 3\text{mil}
\end{align}
$$

Criando as funções

In [0]:
f0 = f_linear([0.06, 0.53, 0.18, 0.18, 0.06], "max")
f1 = f_linear([25, 70, 60, 95, 45], "max")
f2 = f_linear([0, 32.5, 300, 120, 0], "min")
f3 = f_linear([0.1, 0.1, 0.11, 0.35, 0.33], "min")

F = [f0, f1, f2, f3]

E as restrições (já conforme o melhor formato para utilização nos algoritmos de otimização disponíveis)

In [0]:
A_eq = np.array([[1, 1, 1, 1, 1]])
b_eq = np.array([3000])
x0_bounds = (0, 850)
x1_bounds = (0, 220)
x2_bounds = (0, 1300)
x3_bounds = (0, 1615)
x4_bounds = (0, 700)

bounds = [x0_bounds, x1_bounds, x2_bounds, x3_bounds, x4_bounds]

##### Otimização

Como dito anteriormente, para análise dos modelos $\langle X,F \rangle$ é necessário saber os máximo e mínimo valores que podem ser alcançados pela FO dentro de $L$. Como esse exemplo trata de funções lineares, podemos usar a programação linear para isso. Nesse caso, vamos usar uma biblioteca para minimização. Portanto, para maximizar, será necessário a transformação indicada anteriormente.

In [0]:
from scipy.optimize import linprog
import warnings

for f_ in F:
    res = linprog(f_.coefs * -1, A_eq=A_eq, b_eq=b_eq, bounds=bounds, method='simplex')
    if res.success:
        f_.f_max = -1 * res.fun
        f_.x_max = res.x
    else:
        warnings.warn("Otimização mal sucedida.")

In [0]:
for f_ in F:
    res = linprog(f_.coefs, A_eq=A_eq, b_eq=b_eq, bounds=bounds, method='simplex')
    if res.success:
        f_.f_min = res.fun
        f_.x_min = res.x
    else:
        warnings.warn("Otimização mal sucedida.")

Vamos também definir duas funções $\mu_D$: uma para utilizar com bibliotecas para maximização, onde teremos o problema na forma $\max \min$ que queremos originalmente; outra para utilizar com bibliotecas desenvolvidas para minimização, quando teremos um problema $\min \max$, que precisará de ser transformado

In [0]:
def muD_max(F, x):
    x = np.array(x)
    return np.min([f_.mu(x) for f_ in F])

In [0]:
def muD_min(x):
    x = np.array(x)
    return np.max([f_.mu(x)*-1 for f_ in F])

As restrições do tipo $x_1+x_2+x_3+x_4+x_5 = 3\text{mil}$, Ekel chama de condição de balanço. Problemas com esse tipo de restição nos permitem uma soulção muito interessante pois, partindo de um ponto onde $\mu_A = 1$ para qualquer uma das 

In [0]:
import copy as cp

def possivel_subtrair(posicao, limite, passo):
    return True if posicao - passo >= limite[0] else False


def possivel_adicionar(posicao, limite, passo):
    return True if posicao + passo <= limite[1] else False


def caminha(posicao_origem, passo, subtrair_de, adicionar_em):
    posicao_destino = cp.deepcopy(posicao_origem)
    posicao_destino[subtrair_de] -= passo
    posicao_destino[adicionar_em] += passo
    return posicao_destino


def passo_e_melhor(F, posicao_origem, posicao_destino):
    return True if muD_max(F, posicao_destino) > muD_max(F, posicao_origem) else False

In [0]:
def heuristica_relogio(F, posicao_inicial, limites, passo=0.5):
    posicao_atual = np.array(cp.deepcopy(posicao_inicial))
    num_vars = len(posicao_atual)
    i = 0
    while i < num_vars:
        j = 0
        while j < num_vars:
            if i != j:
                if possivel_subtrair(posicao_atual[i], limites[i], passo):
                    if possivel_adicionar(posicao_atual[j], limites[j], passo):
                        if passo_e_melhor(F, posicao_atual, caminha(posicao_atual, passo, i, j)):
                            posicao_atual = caminha(posicao_atual, passo, i, j)
                            #print("posicao atualizada: {}".format(posicao_atual))
                        else:
                            j += 1
                    else:
                        j += 1
                else:
                    i += 1
                    break
            else:
                j += 1
        i += 1
    return posicao_atual

In [19]:
X = heuristica_relogio(F, f1.x_max, bounds, 0.05)
print(X)
print(muD_max(F, X))

[ 850.   220.   609.5 1272.9   47.6]
0.5646304819795885


In [9]:
pip install --upgrade pyswarm

  Stored in directory: /root/.cache/pip/wheels/37/c5/f6/b33b9ac00040cb95c1f00af982a4197334a672d6de43f4699f
Successfully built pyswarm


In [11]:
from pyswarm import pso

lb = [b[0] for b in bounds]
ub = [b[1] for b in bounds]
cons = [lambda x: np.dot(A_eq, x)-b_eq[0]+0.5, lambda x: np.dot(A_eq, x)-b_eq[0]-0.5]

xpt, fopt = pso(muD_min, lb, ub, ieqcons=cons)

Stopping search: Swarm best objective change less than 1e-08


In [12]:
print(xpt)
print(fopt*-1)

[604.42345735 189.28779755 804.24411037 999.84640198 435.9141151 ]
0.5075250162497655
